In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_parquet(r"/content/drive/MyDrive/2_Project_Csv_Group/Combined_Flights_2018.parquet")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5689512 entries, 0 to 637613
Data columns (total 61 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   FlightDate                               datetime64[us]
 1   Airline                                  object        
 2   Origin                                   object        
 3   Dest                                     object        
 4   Cancelled                                bool          
 5   Diverted                                 bool          
 6   CRSDepTime                               int64         
 7   DepTime                                  float64       
 8   DepDelayMinutes                          float64       
 9   DepDelay                                 float64       
 10  ArrTime                                  float64       
 11  ArrDelayMinutes                          float64       
 12  AirTime                           

In [4]:
df.drop(columns=['Marketing_Airline_Network',  'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'DOT_ID_Operating_Airline','Tail_Number', 'OriginAirportSeqID', 'Flight_Number_Operating_Airline', 'OriginCityMarketID','OriginState',  'OriginStateFips',  'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestState','DestStateFips', 'DestWac', 'DivAirportLandings'  ], axis = 0, inplace=True)

In [5]:
cat_cols = ['Airline', 'Origin', 'Dest', 'OriginStateName', 'DestStateName']

for c in cat_cols:
    df[c]=df[c].astype('category')

In [6]:
df['Month'] = df['FlightDate'].dt.month

1. **DepTime**
- We've around `85380` missing values, here we have CRSDeptTIme, but we dont have any values wetre we can calculate DepTime, since other columns which could help derive DepTime are also missing for same viz. `DepDelayMinutes`, `DepDelay`, '`ArrTime`, `ArrDelayMinutes`, `ArrDelayMinutes`, `AirTime` and most impt `AcutalElapsedTime` which could have help us derive all other columns values, but since all of those details are missing.
- After removing for above condition we had some more valeus missing, for other columns, hence here we move for next column
2. **DepDelayMinutes**
- Here this is calculated using (CRSDepTime - DepTime), if te values is in - the it is represented by 0 which means that Flight departed early, else the diff of time is mentioned in minutes, indiccating Flight was delayed in Departure by givem minutes.
- Hence to fill that missing values, we had one logic as mentioned above, we create a confiton to fill whichever possible values where Departure was ealy with 0.
        `df.loc[df['DepDelayMinutes'].isna() & (df['DepTime'] <= df['CRSDepTime']), 'DepDelayMinutes'] = 0.0`
- After imputing with above formula, we're left with 63 missing values, where we don't have any data for mentioned columns `DepDelayMinutes`, `DepDelay`, `ArrTime`, `ArrDelayMinutes`, `AirTime`, `ActualElapsedTime`, `DepDel15`, `DepartureDelayGroups`, `TaxiOut`, `TaxiIn`, `WheelsOff	`, `WheelsOn`, `ArrDelay`, `ArrDel15` and `ArrivalDelayGroups`, nopw all of these parameters are important and dependent on each other to get imputed, hence we need to drop all the rows with all the mentioned values which are null using below condition
        `df = df.dropna(subset=columns_to_check)`

In [7]:
df.dropna(subset=['DepTime'], inplace = True)

In [8]:
df.query("CRSDepTime.notna() and DepTime.notna() and DepDelayMinutes.isna()").sample(3)


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Operating_Airline,IATA_Code_Operating_Airline,OriginAirportID,OriginCityName,OriginStateName,DestCityName,DestStateName,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup
311609,2018-01-11,Endeavor Air Inc.,MSP,GFK,False,False,1310,1310.0,NaN,NaN,1446.0,5.0,50.0,91.0,96.0,284.0,2018,1,1,11,4,9E,9E,13487,"Minneapolis, MN",Minnesota,"Grand Forks, ND",North Dakota,NaN,NaN,1300-1359,38.0,1348.0,1438.0,8.0,1441,5.0,0.0,0.0,1400-1459,2
232439,2018-01-16,Endeavor Air Inc.,DSM,LGA,False,False,755,755.0,NaN,NaN,1154.0,11.0,128.0,168.0,179.0,1031.0,2018,1,1,16,2,9E,9E,11423,"Des Moines, IA",Iowa,"New York, NY",New York,NaN,NaN,0700-0759,43.0,838.0,1146.0,8.0,1143,11.0,0.0,0.0,1100-1159,5
315452,2018-01-02,Endeavor Air Inc.,MSP,GFK,False,False,1327,1327.0,NaN,NaN,1438.0,0.0,52.0,96.0,71.0,284.0,2018,1,1,2,2,9E,9E,13487,"Minneapolis, MN",Minnesota,"Grand Forks, ND",North Dakota,NaN,NaN,1300-1359,15.0,1342.0,1434.0,4.0,1503,-25.0,0.0,-2.0,1500-1559,2


In [9]:
df.loc[df['DepDelayMinutes'].isna() & (df['DepTime'] <= df['CRSDepTime']), 'DepDelayMinutes'] = 0.0

In [10]:
columns_to_check = [
    "DepDelayMinutes", "DepDelay", "ArrTime", "ArrDelayMinutes", "AirTime",
    "ActualElapsedTime", "DepDel15", "DepartureDelayGroups", "TaxiOut", "TaxiIn",
    "WheelsOff", "WheelsOn", "ArrDelay", "ArrDel15", "ArrivalDelayGroups"
]
df = df.dropna(subset=columns_to_check)


In [11]:
df.isna().sum()

,0
FlightDate,0
Airline,0
Origin,0
Dest,0
Cancelled,0
Diverted,0
CRSDepTime,0
DepTime,0
DepDelayMinutes,0
DepDelay,0


In [21]:
grouped_df = df.groupby(['Airline', 'Dest', 'Origin'], as_index=False).agg({
    'TaxiIn': 'mean',
    'TaxiOut': 'mean',
    'WheelsOff': 'mean',
    'WheelsOn': 'mean'
})


<ipython-input-21-d33930ed2a20>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = df.groupby(['Airline', 'Dest', 'Origin'], as_index=False).agg({


ValueError: Length of values (14225) does not match length of index (3833200)

In [14]:
# Compute averages for each (Airline, Origin, Destination) combination
taxi_times_avg = df.groupby(['Airline', 'Origin', 'Dest'])[['TaxiIn', 'TaxiOut', 'WheelsOff', 'WheelsOn']].mean().reset_index()
taxi_times_avg.head()
# Save this as a reference CSV for later use
# taxi_times_avg.to_csv("taxi_times_lookup.csv", index=False)


<ipython-input-14-e87953779ec6>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  taxi_times_avg = df.groupby(['Airline', 'Origin', 'Dest'])[['TaxiIn', 'TaxiOut', 'WheelsOff', 'WheelsOn']].mean().reset_index()


,Airline,Origin,Dest,TaxiIn,TaxiOut,WheelsOff,WheelsOn
0,Air Wisconsin Airlines Corp,ABE,ABE,NaN,NaN,NaN,NaN
1,Air Wisconsin Airlines Corp,ABE,ABI,NaN,NaN,NaN,NaN
2,Air Wisconsin Airlines Corp,ABE,ABQ,NaN,NaN,NaN,NaN
3,Air Wisconsin Airlines Corp,ABE,ABR,NaN,NaN,NaN,NaN
4,Air Wisconsin Airlines Corp,ABE,ABY,NaN,NaN,NaN,NaN


In [17]:
taxi_times_avg[taxi_times_avg[['TaxiIn', 'TaxiOut', 'WheelsOff', 'WheelsOn']].notna().all(axis=1)]


,Airline,Origin,Dest,TaxiIn,TaxiOut,WheelsOff,WheelsOn
254,Air Wisconsin Airlines Corp,ABE,ORD,11.311518,16.510471,1156.416230,1226.968586
5345,Air Wisconsin Airlines Corp,ALB,IAD,5.194030,12.014925,1946.014925,1998.977612
5434,Air Wisconsin Airlines Corp,ALB,ORD,17.439252,14.336449,1164.429907,1259.439252
8602,Air Wisconsin Airlines Corp,ATW,DEN,14.873786,15.160194,851.165049,968.286408
8764,Air Wisconsin Airlines Corp,ATW,ORD,15.426716,15.368584,1070.158318,1137.466914
...,...,...,...,...,...,...,...
3814227,Virgin America,SFO,PHL,10.440476,21.345238,2004.011905,886.214286
3814243,Virgin America,SFO,PSP,5.210714,16.692857,1683.960714,1782.232143
3814252,Virgin America,SFO,RDU,6.101124,22.730337,2249.730337,626.382022
3814265,Virgin America,SFO,SAN,3.845824,17.451820,1430.608137,1540.423983


In [20]:
taxi_times_avg['Dest'].unique()

['ABE', 'ABI', 'ABQ', 'ABR', 'ABY', ..., 'XNA', 'YAK', 'YKM', 'YNG', 'YUM']
Length: 370
Categories (370, object): ['ABE', 'ABI', 'ABQ', 'ABR', ..., 'YAK', 'YKM', 'YNG', 'YUM']

In [19]:
taxi_times_avg['Origin'].unique()

['ABE', 'ABI', 'ABQ', 'ABR', 'ABY', ..., 'XNA', 'YAK', 'YKM', 'YNG', 'YUM']
Length: 370
Categories (370, object): ['ABE', 'ABI', 'ABQ', 'ABR', ..., 'YAK', 'YKM', 'YNG', 'YUM']

In [18]:
taxi_times_avg['Airline'].unique()

['Air Wisconsin Airlines Corp', 'Alaska Airlines Inc.', 'Allegiant Air', 'American Airlines Inc.', 'Cape Air', ..., 'Southwest Airlines Co.', 'Spirit Air Lines', 'Trans States Airlines', 'United Air Lines Inc.', 'Virgin America']
Length: 28
Categories (28, object): ['Air Wisconsin Airlines Corp', 'Alaska Airlines Inc.', 'Allegiant Air',
                          'American Airlines Inc.', ..., 'Spirit Air Lines', 'Trans States Airlines',
                          'United Air Lines Inc.', 'Virgin America']

In [18]:
taxi_times_avg.to_csv('Taxi_AvgValues.csv', index=False)

In [19]:
taxi_times_avg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45998400 entries, 0 to 45998399
Data columns (total 8 columns):
 #   Column     Dtype   
---  ------     -----   
 0   Month      int32   
 1   Airline    category
 2   Origin     category
 3   Dest       category
 4   TaxiIn     float64 
 5   TaxiOut    float64 
 6   WheelsOff  float64 
 7   WheelsOn   float64 
dtypes: category(3), float64(4), int32(1)
memory usage: 1.8 GB


In [15]:
taxi_times_avg[taxi_times_avg['TaxiIn'].notna()]

,Month,Airline,Origin,Dest,TaxiIn,TaxiOut,WheelsOff,WheelsOn
8764,1,Air Wisconsin Airlines Corp,ATW,ORD,13.054264,17.387597,1113.627907,1185.062016
9874,1,Air Wisconsin Airlines Corp,AVP,ORD,12.666667,22.166667,1883.833333,1969.833333
10614,1,Air Wisconsin Airlines Corp,AZO,ORD,11.200000,20.350000,1271.012500,1225.750000
20515,1,Air Wisconsin Airlines Corp,BUF,IAD,9.500000,12.500000,1688.500000,1776.000000
20604,1,Air Wisconsin Airlines Corp,BUF,ORD,9.500000,16.500000,1315.000000,1367.500000
...,...,...,...,...,...,...,...,...
45853678,12,United Air Lines Inc.,TPA,SFO,9.581395,15.744186,802.465116,1060.372093
45854562,12,United Air Lines Inc.,TUL,DEN,7.052632,12.000000,615.789474,638.421053
45854637,12,United Air Lines Inc.,TUL,IAH,10.285714,16.142857,785.142857,899.428571
45854932,12,United Air Lines Inc.,TUS,DEN,8.695652,12.565217,1293.521739,1457.391304
